# Answers to Exercises

In [15]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext lab_black
import json

from fastai.tabular.all import *

import pandas as pd

import kglab
import rdflib
import requests
import helpers
import widgets

# Introduction and Setup for Sinopia's Knowledge Graph
## Exercise 1
Compare the total number of triples for National Library of Medicine in each Sinopia environment; development, stage, and production.

In [4]:
dev_nlm = helpers.create_kg(
    "https://api.development.sinopia.io/resource?group=nlm",
    name="NLM Sinopia Development",
)
stage_nlm = helpers.create_kg(
    "https://api.stage.sinopia.io/resource?group=nlm", name="NLM Sinopia Stage"
)
prod_nlm = helpers.create_kg(
    "https://api.sinopia.io/resource?group=nlm", name="NLM Sinopia Production"
)

0....100....2000....100....200....300....400..

In [9]:
print(
    f"""Number of Triples for NLM per Environment:
Development: {len(dev_nlm.rdf_graph()):,}
      Stage: {len(stage_nlm.rdf_graph()):,}
 Production: {len(prod_nlm.rdf_graph()):,}"""
)

Number of Triples for NLM per Environment:
Development: 0
      Stage: 7,969
 Production: 12,046


# Analysis and Visualization of Sinopia Graphs
## Exercise 1
Using any of the three Sinopia environments (development, stage, or production), find a BIBFRAME Work with a corresponding BIBFRAME Instance and then replicate the steps above to create a Sinopia BIBFRAME Knowledge Graph.

For this exercise, I am using this BIBFRAME work from production, https://api.sinopia.io/resource/07d0f7e4-244d-40ed-8c83-537b495eee0b, that has an embedded BIBFRAME Instance as a blank node.

In [16]:
work_instance_result = requests.get(
    "https://api.sinopia.io/resource/07d0f7e4-244d-40ed-8c83-537b495eee0b"
)
bf_graph = rdflib.Graph()
for ns, url in helpers.NAMESPACES.items():
    bf_graph.namespace_manager.bind(ns, url)
bf_graph.parse(
    data=json.dumps(work_instance_result.json().get("data")), format="json-ld"
)

<Graph identifier=N782f29d9e8744ac683d28d6325d72a5a (<class 'rdflib.graph.Graph'>)>

In [19]:
print(bf_graph.serialize(format="turtle").decode())

@prefix bf: <http://id.loc.gov/ontologies/bibframe/> .
@prefix bflc: <http://id.loc.gov/ontologies/bflc/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sinopia: <http://sinopia.io/vocabulary/> .

<https://api.sinopia.io/resource/07d0f7e4-244d-40ed-8c83-537b495eee0b> a bf:Work ;
    bf:Work <http://share-vde.org/sharevde/rdfBibframe/Work/13390539-1> ;
    bf:classification [ a bf:ClassificationLcc ;
            bf:classificationPortion "B2430.R272"@eng,
                "HD4851"@eng ] ;
    bf:content <http://id.loc.gov/vocabulary/contentTypes/txt> ;
    bf:contribution [ a bf:Contribution ;
            bf:agent [ a bf:Person ;
                    rdf:value "http://id.loc.gov/authorities/names/n81053868" ] ;
            bf:role <http://id.loc.gov/vocabulary/relators/aut> ] ;
    bf:expressionOf [ a bf:Work ;
            bf:Work <http://share-vde.org/sharevde/rdfBibframe/Work/13390539> ] ;
    bf:hasInstance 

In [18]:
bf_kg = kglab.KnowledgeGraph(
    import_graph=bf_graph, name="BIBFRAME Work and Instance graph"
)
measure = kglab.Measure()
measure.measure_graph(bf_kg)

print("Edges: {:,}\n".format(measure.get_edge_count()))
print("Nodes: {:,}\n".format(measure.get_node_count()))

Edges: 101

Nodes: 51



# Natural Language Processing (NLP) with spaCy
## Exercise 1

# HuggingFace Transformers
## Exercise 1

# FastAI with PyTorch
## Exercise 1